In [ ]:
import requests
import pandas as pd

def fetch_api_data_to_df(base_url, params={}, data_key='data', max_pages=None, verbose=False):
    """
    Fetch paginated API data and return a flattened DataFrame.
    
    Parameters:
        base_url (str): Base URL of the API.
        params (dict): Additional parameters (e.g., {'per_page': 6}).
        data_key (str): Key that contains the main list of data (default 'data').
        max_pages (int or None): Max pages to retrieve; None fetches all.
        verbose (bool): Print page-by-page status.

    Returns:
        pd.DataFrame: Combined DataFrame from all pages.
    """
    combined_data = []
    page = 1

    while True:
        params['page'] = page
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an error for bad responses
        json_data = response.json()

        if data_key not in json_data:
            raise KeyError(f"'{data_key}' not found in response JSON.")

        page_data = json_data[data_key]
        if verbose:
            print(f"Fetched page {page}: {len(page_data)} records")

        combined_data.extend(page_data)

        total_pages = json_data.get('total_pages', 1)
        if max_pages is not None:
            total_pages = min(total_pages, max_pages)

        if page >= total_pages:
            break

        page += 1

    return pd.DataFrame(combined_data)
